<a href="https://colab.research.google.com/github/seifahmed2004/Information-retrieval-project-/blob/main/IR_project_seif_ahmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-terrier --quiet
!pip install nltk --quiet
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git --quiet
!pip install transformers
!pip install flair
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

  Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
fatal: destination path 'terrier-prf' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
maven is already the newest version (3.6.3-5).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/content/terrier-prf
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< org.terrier:terrier-prf >-----------------------
[INFO] Building terrier-prf 0.2-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ terrier-prf ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] skip non existing resourceDirectory /content/terrier-prf/src/main/resources
[INFO] 
[INFO] --- ma

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import *
from nltk.stem.porter import *
from flair.data import Sentence
from flair.embeddings import WordEmbeddings
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf
import tensorflow_hub as hub
import gensim
from gensim.models import Word2Vec
pd.set_option('display.max_colwidth', 150)
import pyterrier as pt
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [3]:
data = pd.read_csv("/content/emotion_sentimen_dataset (1).csv")
data

,ID,text,Emotion
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate
1,1,im so full of life i feel appalled,neutral
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral
...,...,...,...
839550,839550,i feel like telling these horny devils to find a site more suited to that sort of interest the playboy if there is one forum perhaps,neutral
839551,839551,i began to realize that when i was feeling agitated or restless that i would have a thought to go do the dishes,neutral
839552,839552,i feel very curious be why previous early dawn in the time here don t you seek his trouble,neutral
839553,839553,i feel that becuase of the tyranical nature of the government in el salvador and the savage social conditions of the general people that the set o...,neutral


In [4]:
nltk.download('stopwords')
print(stopwords.words('english'))
nltk.download('stopwords')
nltk.download('punkt')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
# Function to remove stopwords
def remove_stopwords(text):

    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words] #Lower is used to normalize al the words make them in lower case
    print('Tokens are:',tokens,'\n')
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'content' column
data['processed_text'] = data['text'].apply(remove_stopwords)

Streaming output truncated to the last 5000 lines.
Tokens are: ['i', 'start', 'to', 'feel', 'that', 'fearful', 'urgency', 'i', 'know', 'he', 'is', 'speaking', 'to', 'me', 'and', 'ive', 'learned', 'to', 'recognize', 'it', 'and', 'ive', 'learned', 'that', 'being', 'obedient', 'and', 'walking', 'back', 'to', 'my', 'bible', 'is', 'a', 'key', 'factor'] 

Tokens are: ['i', 'feel', 'it', 'so', 'easily', 'like', 'that', 'of', 'a', 'gentle', 'rain', 'that', 'warms', 'the', 'earth', 'and', 'brings', 'laughter', 'and', 'delight', 'from', 'all', 'those', 'that', 'pause', 'to', 'take', 'notice', 'of', 'such', 'a', 'blessing'] 

Tokens are: ['i', 'approached', 'him', 'hesitantly', 'fearing', 'that', 'he', 'd', 'respond', 'in', 'a', 'way', 'that', 'would', 'make', 'me', 'feel', 'more', 'isolated'] 

Tokens are: ['i', 'realise', 'how', 'blessed', 'i', 'am', 'to', 'have', 'the', 'opportunity', 'to', 'go', 'to', 'a', 'country', 'that', 'contains', 'so', 'much', 'of', 'the', 'world', 's', 'history', 'and

In [45]:
print("df after processing:\n")
df

df after processing:



,qid,Text,docno
0,1,measurement of dielectric constant of liquids by the use of microwave techniques,0
1,2,mathematical analysis and design details of waveguide fed microwave radiations,1
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics,2
3,4,systems of data coding for information transfer,3
4,5,use of programs in engineering testing of computers,4
...,...,...,...
88,89,tunnel diode construction and its electrical characteristics explained,88
89,90,electronic density of states at the surface of a semiconductor compared with that at depth,89
90,91,resistivity of metallic thin films related to surface roughness,90
91,92,the phenomenon of radiation caused by charged particles moving in varying electric and magnetic fields,91


In [7]:
stemmer = PorterStemmer()

In [8]:
def Steem_text(text):

    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

In [9]:
#a function to clean the documents
def clean(text):
   text = re.sub(r"http\S+", " ", text) # remove urls
   text = re.sub(r"RT ", " ", text) # remove rt
   text = re.sub(r"@[\w]*", " ", text) # remove handles
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump
   text = re.sub(r"\s+", " ", text) # remove extra white space
   text = text.strip()
   return text
data['processed_text'] = data['processed_text'].apply(clean)
data

,ID,text,Emotion,processed_text
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,seriously hate one subject death feel reluctant drop
1,1,im so full of life i feel appalled,neutral,im full life feel appalled
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feelings think afraid accept possibility might make
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive really angry r feel like idiot trusting first place
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspicious one outside like rapture happened something
...,...,...,...,...
839550,839550,i feel like telling these horny devils to find a site more suited to that sort of interest the playboy if there is one forum perhaps,neutral,feel like telling horny devils find site suited sort interest playboy one forum perhaps
839551,839551,i began to realize that when i was feeling agitated or restless that i would have a thought to go do the dishes,neutral,began realize feeling agitated restless would thought go dishes
839552,839552,i feel very curious be why previous early dawn in the time here don t you seek his trouble,neutral,feel curious previous early dawn time seek trouble
839553,839553,i feel that becuase of the tyranical nature of the government in el salvador and the savage social conditions of the general people that the set o...,neutral,feel becuase tyranical nature government el salvador savage social conditions general people set thought run shoe shop miguel marmols testimoney a...


In [10]:
data['processed_text']= data['processed_text'].apply(Steem_text)
data

,ID,text,Emotion,processed_text
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,serious hate one subject death feel reluct drop
1,1,im so full of life i feel appalled,neutral,im full life feel appal
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feel think afraid accept possibl might make
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive realli angri r feel like idiot trust first place
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspici one outsid like raptur happen someth
...,...,...,...,...
839550,839550,i feel like telling these horny devils to find a site more suited to that sort of interest the playboy if there is one forum perhaps,neutral,feel like tell horni devil find site suit sort interest playboy one forum perhap
839551,839551,i began to realize that when i was feeling agitated or restless that i would have a thought to go do the dishes,neutral,began realiz feel agit restless would thought go dish
839552,839552,i feel very curious be why previous early dawn in the time here don t you seek his trouble,neutral,feel curiou previou earli dawn time seek troubl
839553,839553,i feel that becuase of the tyranical nature of the government in el salvador and the savage social conditions of the general people that the set o...,neutral,feel becuas tyran natur govern el salvador savag social condit gener peopl set thought run shoe shop miguel marmol testimoney abl work


In [11]:
data['docno'] = data['ID'].astype(str)
data

,ID,text,Emotion,processed_text,docno
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,serious hate one subject death feel reluct drop,0
1,1,im so full of life i feel appalled,neutral,im full life feel appal,1
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feel think afraid accept possibl might make,2
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive realli angri r feel like idiot trust first place,3
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspici one outsid like raptur happen someth,4
...,...,...,...,...,...
839550,839550,i feel like telling these horny devils to find a site more suited to that sort of interest the playboy if there is one forum perhaps,neutral,feel like tell horni devil find site suit sort interest playboy one forum perhap,839550
839551,839551,i began to realize that when i was feeling agitated or restless that i would have a thought to go do the dishes,neutral,began realiz feel agit restless would thought go dish,839551
839552,839552,i feel very curious be why previous early dawn in the time here don t you seek his trouble,neutral,feel curiou previou earli dawn time seek troubl,839552
839553,839553,i feel that becuase of the tyranical nature of the government in el salvador and the savage social conditions of the general people that the set o...,neutral,feel becuas tyran natur govern el salvador savag social condit gener peopl set thought run shoe shop miguel marmol testimoney abl work,839553


In [12]:
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
index_ref = indexer.index(data["processed_text"], data["docno"])
print(index_ref.toString())

19:27:08.565 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (5965) - further warnings are suppressed
19:36:00.916 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 42 empty documents
./DatasetIndex/data.properties


In [13]:
#we will first load the index
index = pt.IndexFactory.of(index_ref)
#we will call getCollectionStatistics() to check the stats
print(index.getCollectionStatistics().toString())

Number of documents: 839555
Number of terms: 50306
Number of postings: 6812406
Number of fields: 0
Number of tokens: 7088758
Field names: []
Positions:   false



In [14]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString()))

Streaming output truncated to the last 5000 lines.
triplet -> term17152 Nt=2 TF=2 maxTF=2147483647 @{0 10922058 2} 
tripletsplu -> term43016 Nt=2 TF=2 maxTF=2147483647 @{0 10922067 0} 
tripli -> term26550 Nt=4 TF=4 maxTF=2147483647 @{0 10922076 4} 
triplic -> term50215 Nt=2 TF=2 maxTF=2147483647 @{0 10922094 6} 
tripll -> term8589 Nt=8 TF=8 maxTF=2147483647 @{0 10922104 2} 
tripod -> term20257 Nt=18 TF=18 maxTF=2147483647 @{0 10922137 4} 
tripoli -> term31484 Nt=6 TF=6 maxTF=2147483647 @{0 10922203 4} 
trippel -> term20499 Nt=2 TF=2 maxTF=2147483647 @{0 10922229 0} 
tripper -> term28673 Nt=6 TF=6 maxTF=2147483647 @{0 10922238 0} 
trippi -> term31258 Nt=8 TF=8 maxTF=2147483647 @{0 10922265 0} 
trippin -> term40775 Nt=2 TF=2 maxTF=2147483647 @{0 10922298 4} 
trippl -> term25335 Nt=4 TF=4 maxTF=2147483647 @{0 10922308 0} 
triptr -> term8202 Nt=4 TF=4 maxTF=2147483647 @{0 10922324 0} 
triptych -> term8765 Nt=4 TF=4 maxTF=2147483647 @{0 10922340 6} 
trisomi -> term39834 Nt=2 TF=2 maxTF=2147

In [15]:
index.getLexicon()["result"].getDocumentFrequency()

2458

In [16]:
x = index.getDirectIndex()
y = index.getDocumentIndex()
z = index.getLexicon()
a = 0
for posting in x.getPostings(y.getDocumentEntry(a)):
    termid = posting.getId()
    B = z.getLexiconEntry(termid)
    print("%s with frequency %d" % (B.getKey(),posting.getFrequency()))

hate with frequency 1
reluct with frequency 1
subject with frequency 1
death with frequency 1
drop with frequency 1
feel with frequency 1


In [17]:
def preprocess(sentence):
    sentence = remove_stopwords(sentence)
    sentence = clean(sentence)
    sentence = Steem_text(sentence)
    return sentence

In [18]:
# Preprocess user query
query = "fun"
query = preprocess(query)

Tokens are: ['fun'] 



In [19]:
# Set up retrieval model with TF-IDF as weighting model
tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)

# Search for relevant documents using the retrieval model
results = tfidf_retr.search(query)
results

,qid,docid,docno,rank,score,query
0,1,295451,295451,0,6.881112,fun
1,1,712260,712260,1,6.881112,fun
2,1,292699,292699,2,6.414431,fun
3,1,709508,709508,3,6.414431,fun
4,1,6419,6419,4,6.031474,fun
5,1,113446,113446,5,6.031474,fun
6,1,154301,154301,6,6.031474,fun
7,1,172715,172715,7,6.031474,fun
8,1,186026,186026,8,6.031474,fun
9,1,262359,262359,9,6.031474,fun


In [20]:
# Retrieve top relevant documents based on the query
top_answer = data[data["docno"].isin(['6419', '48097'])]['processed_text']
top_answer

6419                                                                     fun feel passion ever
48097    go back energi want feel take bodi amaz fun thought fun solut fun activ movement come
Name: processed_text, dtype: object

In [21]:
# Calculate frequency of the term 'empty' in the top related documents
freq_of_items = []
for i in range(len(top_answer)):
    tweet = top_answer.iloc[i].split()
    counter = 0
    for word_in_tweet in tweet:
        if word_in_tweet == 'fun':
            counter += 1
    freq_of_items.append(counter)
print('The frequency of the item in the top related docs are:', freq_of_items)

The frequency of the item in the top related docs are: [1, 3]


In [22]:
index = pt.IndexFactory.of(index_ref)

In [23]:
# Define our retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)
results = bm25.search(query)
results

,qid,docid,docno,rank,score,query
0,1,295451,295451,0,12.599108,fun
1,1,712260,712260,1,12.599108,fun
2,1,292699,292699,2,11.744628,fun
3,1,709508,709508,3,11.744628,fun
4,1,6419,6419,4,11.043445,fun
5,1,113446,113446,5,11.043445,fun
6,1,154301,154301,6,11.043445,fun
7,1,172715,172715,7,11.043445,fun
8,1,186026,186026,8,11.043445,fun
9,1,262359,262359,9,11.043445,fun


In [24]:
data[['text']][data['docno'].isin(results['docno'].loc[0:4].tolist())]

,text
6419,i have fun i feel as passionate about it as ever
292699,i don t object to the fun i ve been having or the extent of the fun but more of the feeling of needing to do all of those things when unhappy
295451,i feel pressured to do fun things or because i am going through the motions of what a fun involved mom should do but because i am just having fun ...
709508,i don t object to the fun i ve been having or the extent of the fun but more of the feeling of needing to do all of those things when unhappy
712260,i feel pressured to do fun things or because i am going through the motions of what a fun involved mom should do but because i am just having fun ...


In [25]:
rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)

rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]

expanded_query

'applypipeline:off carefr^0.020895524 bless^0.020895524 fun^0.802985072 need^0.017910447 sociabl^0.020895524 selfish^0.020895524 passion^0.041791048 object^0.017910447 unhappi^0.017910447 extent^0.017910447'

In [26]:
# Just print the expanded query with term scores
for s in expanded_query.split()[1:]:
  print(s)

print("\n" + query)

carefr^0.020895524
bless^0.020895524
fun^0.802985072
need^0.017910447
sociabl^0.020895524
selfish^0.020895524
passion^0.041791048
object^0.017910447
unhappi^0.017910447
extent^0.017910447

fun


In [27]:
# After that you can search using the expanded query
expanded_query_formatted = ' '.join(expanded_query.split()[1:])

results_wqe = bm25.search(expanded_query_formatted)

print("   Before Expansion    After Expansion")
print(pd.concat([results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna(''))


#Let's check the tweets text for the top 5 retrieved tweets
data[['text']][data['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

   Before Expansion    After Expansion
   docid_1    score_1  docid_2    score_2
0   295451  12.599108   292699  12.677847
1   712260  12.599108   709508  12.677847
2   292699  11.744628   295451  12.599108
3   709508  11.744628   712260  12.599108
4     6419  11.043445     6419  11.598789


,text
6419,i have fun i feel as passionate about it as ever
262359,i have fun i feel as passionate about it as ever
292699,i don t object to the fun i ve been having or the extent of the fun but more of the feeling of needing to do all of those things when unhappy
295451,i feel pressured to do fun things or because i am going through the motions of what a fun involved mom should do but because i am just having fun ...
709508,i don t object to the fun i ve been having or the extent of the fun but more of the feeling of needing to do all of those things when unhappy
712260,i feel pressured to do fun things or because i am going through the motions of what a fun involved mom should do but because i am just having fun ...


In [28]:
!pip install flask_ngrok
!pip install pyngrok

In [29]:
df2 = data.head(50)

df2 = df2.to_dict()

df2

{'ID': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49},
 'text': {0: 'i seriously hate one subject to death but now i feel reluctant to drop it',
  1: 'im so full of life i feel appalled',
  2: 'i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it',
  3: 'ive been really angry with r and i feel like an idiot for trusting him in the first place',
  4: 'i feel suspicious if there is no one outside like the rapture has happened or something',
  5: 'i feel jealous becasue i wanted that kin

In [30]:
def sui(df2 , que):
 i = 0

 quer = preprocess(que)

 docs_id = []

 for key, value in df2.items():
   if key == 'processed_text':
         val = value.values()
         for doc in val:
           terms = doc.split()
           for term in terms:
             if term == quer and i not in docs_id:
               docs_id.append(f'''Document number {i} -----> \n{data["text"][i]}''')
           i = i + 1
 return docs_id

In [31]:
query2 = "like"
X = sui(df2 , query2)
X

Tokens are: ['like'] 



['Document number 3 -----> \nive been really angry with r and i feel like an idiot for trusting him in the first place',
 'Document number 4 -----> \ni feel suspicious if there is no one outside like the rapture has happened or something',
 'Document number 9 -----> \ni feel like i ve regained another vital part of my life which is living',
 'Document number 10 -----> \ni feel a bit like franz liebkind in the producers not many people know it but the fuhrer was a terrific dancer',
 'Document number 14 -----> \ni just try not to talk to anyone when i feel irritable like that',
 'Document number 15 -----> \ni feel like they hated me since then',
 'Document number 19 -----> \ni feel like i have to redeem myself even though i think they realized why i was distraught and were ok with it',
 'Document number 33 -----> \ni end up getting unwanted attention from boys i want little to do with or ill be sort of starting something with a boy then find myself flirtiing with others in his presence o

In [37]:
from google.colab.output import eval_js
print (eval_js("google.colab.kernel.proxyPort(5000)"))

https://nwd7jb824mi-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [38]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok

# Assuming you've already defined the sui function and imported necessary modules

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return """
    <style>
        body {
            background-color: 	#f1c232;
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }

        .header {
            background-color:		#f1c232;
            color: black ;
            padding: 20px 0;

        }

        .container {
            text-align: center;
        }

        h1 {
            text-align: center;
            margin:0;
            padding: 10px 0;
        }

        #searchInput {
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 50px; /* Increased border-radius for a rounded appearance */
            margin-bottom: 10px;
            width: 300px; /* Adjust the width as needed */
            box-sizing: border-box; /* Include padding and border in the element's total width */
            transition: border-color 0.5s; /* Smooth transition for border color change */
        }

        #searchInput:focus {
            border-color: #403072; /* Change border color on focus */
        }

        button {
            padding: 10px 20px;
            background-color:#ce7e00;
            color: white;
            border: none;
            border-radius: 20px; /* Increased border-radius for a rounded appearance */
            cursor: pointer;
            transition: background-color 0.5s; /* Smooth transition for background color change */
        }

        button:hover {
            background-color: #7d73af; /* Change background color on hover */
        }

          #searchResult {
            text-align: 10 px; /* Center the search results */
            color: black; /* Set the color of search results */
            margin: 10 px;
            padding: 10px
        }
    </style>

    <div class="header">
        <h1>Search Engine</h1>
    </div>
    <div class="container">
        <input type="text" id="searchInput" placeholder="search...">
        <button onclick="search()">Search</button>
    </div>
    <div id="searchResult"></div>

    <script>
        function search() {
            var searchTerm = document.getElementById("searchInput").value;
            fetch('/search', {
                method: 'POST',
                body: JSON.stringify({ query: searchTerm }),
                headers:{
                    'Content-Type': 'application/json'
                }
            })
            .then(response => response.json())
            .then(data => {
                console.log("Received data:", data); // Debug: Check if data is received
                var resultDiv = document.getElementById("searchResult");
                resultDiv.innerHTML = "<h2>Relevant Documents IDs:</h2>";
                if (data.results.length === 0) {
                    resultDiv.innerHTML += "<p>No documents found</p>";
                } else {
                    data.results.forEach(doc => {
                        console.log("Displaying document:", doc); // Debug: Check if document is displayed
                        resultDiv.innerHTML += "<p>" + doc + "</p>";
                    });
                }
            })
            .catch(error => {
                console.error('Error occurred during fetch:', error); // Debug: Log fetch errors
            });
        }
    </script>
    """

@app.route("/search", methods=['POST'])
def search():
    query = request.json['query']
    print("Received query:", query)  # Debug: Check if Flask receives the query
    results = sui(df2, query)
    print("Search results:", results)  # Debug: Check if sui function returns results
    return {'results': results}

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 715, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packag

Received query: hello
Tokens are: ['hello'] 

Search results: []


INFO:werkzeug:127.0.0.1 - - [11/May/2024 19:39:22] "POST /search HTTP/1.1" 200 -


Received query: sad
Tokens are: ['sad'] 

Search results: []


INFO:werkzeug:127.0.0.1 - - [11/May/2024 19:39:23] "POST /search HTTP/1.1" 200 -


Received query: sad
Tokens are: ['sad'] 

Search results: []


INFO:werkzeug:127.0.0.1 - - [11/May/2024 19:39:37] "POST /search HTTP/1.1" 200 -


Received query: feel
Tokens are: ['feel'] 

Search results: ['Document number 0 -----> \ni seriously hate one subject to death but now i feel reluctant to drop it', 'Document number 1 -----> \nim so full of life i feel appalled', 'Document number 2 -----> \ni sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it', 'Document number 3 -----> \nive been really angry with r and i feel like an idiot for trusting him in the first place', 'Document number 4 -----> \ni feel suspicious if there is no one outside like the rapture has happened or something', 'Document number 5 -----> \ni feel jealous becasue i wanted that kind of love the true connection between two souls and i wanted that', 'Document number 7 -----> \ni finally fell asleep feeling angry useless and still full of anxiety', 'Document number 8 -----> \ni feel a bit annoyed and antsy in a good way', 'Document number 9 -----> \ni feel like i ve regained anoth

In [41]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")

df= vaswani_dataset.get_topics()

df['docno'] = df.index

# Rename column 'A' to 'X'
df = df.rename(columns={'query': 'Text'})

qrels = vaswani_dataset.get_qrels()

qrels['docno']=qrels['docno'].astype(str)

df

qrels:   0%|          | 0.00/6.63k [00:00<?, ?iB/s]

,qid,Text,docno
0,1,measurement of dielectric constant of liquids by the use of microwave techniques,0
1,2,mathematical analysis and design details of waveguide fed microwave radiations,1
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics,2
3,4,systems of data coding for information transfer,3
4,5,use of programs in engineering testing of computers,4
...,...,...,...
88,89,tunnel diode construction and its electrical characteristics explained,88
89,90,electronic density of states at the surface of a semiconductor compared with that at depth,89
90,91,resistivity of metallic thin films related to surface roughness,90
91,92,the phenomenon of radiation caused by charged particles moving in varying electric and magnetic fields,91


In [42]:
indexref2 = vaswani_dataset.get_index()
index2 = pt.IndexFactory.of(indexref2)

print(index2.getCollectionStatistics().toString())

data.direct.bf:   0%|          | 0.00/388k [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/234k [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/362k [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/682k [00:00<?, ?iB/s]

data.lexicon.fsomaphash:   0%|          | 0.00/777 [00:00<?, ?iB/s]

...

data.meta.idx:   0%|          | 0.00/89.3k [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/224k [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/4.29k [00:00<?, ?iB/s]

md5sums:   0%|          | 0.00/619 [00:00<?, ?iB/s]

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 1
Number of tokens: 271581
Field names: [text]
Positions:   false



In [43]:
retr = pt.BatchRetrieve(index2, controls = {"wmodel": "TF_IDF"})

res = retr.search("mathematical")
res

,qid,docid,docno,rank,score,query
0,1,4746,4747,0,5.168347,mathematical
1,1,7399,7400,1,5.036916,mathematical
2,1,5629,5630,2,4.912003,mathematical
3,1,7997,7998,3,4.912003,mathematical
4,1,4546,4547,4,4.679886,mathematical
...,...,...,...,...,...,...
147,1,3484,3485,147,1.828498,mathematical
148,1,7283,7284,148,1.747822,mathematical
149,1,6714,6715,149,1.702745,mathematical
150,1,8622,8623,150,1.606095,mathematical


In [44]:
eval = pt.Evaluate(res,qrels)
eval

{'map': 4.7960250544348844e-06, 'ndcg': 0.00022891881462746983}